In [2]:
import pandas as pd
import numpy as np 
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from torchsummary import summary
from torch.autograd import Variable

In [3]:
print('GPU:', torch.cuda.is_available())

GPU: False


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty

/bin/bash: google-drive-ocamlfuse: command not found


# Load Data

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Read the data
import glob,os
# path=r'../Dataset/'
path=r'../Dataset/'
file=glob.glob(os.path.join(path, "data*.csv"))
print(file)
dl= []
for f in file:
 dl.append(pd.read_csv(f,header=[0],index_col=None))
df=pd.concat(dl)

['/content/drive/MyDrive/Colab Notebooks/Dataset/data1.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data2.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data3.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data4.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data5.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data6.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data8.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data9.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data10.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data11.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data12.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data13.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data14.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data15.csv', '/content/drive/MyDrive/Colab Notebooks/Dataset/data7.csv']


In [7]:
df.shape

(78377, 129)

In [8]:
df.head()

,R1-PA1:VH,R1-PM1:V,R1-PA2:VH,R1-PM2:V,R1-PA3:VH,R1-PM3:V,R1-PA4:IH,R1-PM4:I,R1-PA5:IH,R1-PM5:I,R1-PA6:IH,R1-PM6:I,R1-PA7:VH,R1-PM7:V,R1-PA8:VH,R1-PM8:V,R1-PA9:VH,R1-PM9:V,R1-PA10:IH,R1-PM10:I,R1-PA11:IH,R1-PM11:I,R1-PA12:IH,R1-PM12:I,R1:F,R1:DF,R1-PA:Z,R1-PA:ZH,R1:S,R2-PA1:VH,R2-PM1:V,R2-PA2:VH,R2-PM2:V,R2-PA3:VH,R2-PM3:V,R2-PA4:IH,R2-PM4:I,R2-PA5:IH,R2-PM5:I,R2-PA6:IH,...,R4-PA2:VH,R4-PM2:V,R4-PA3:VH,R4-PM3:V,R4-PA4:IH,R4-PM4:I,R4-PA5:IH,R4-PM5:I,R4-PA6:IH,R4-PM6:I,R4-PA7:VH,R4-PM7:V,R4-PA8:VH,R4-PM8:V,R4-PA9:VH,R4-PM9:V,R4-PA10:IH,R4-PM10:I,R4-PA11:IH,R4-PM11:I,R4-PA12:IH,R4-PM12:I,R4:F,R4:DF,R4-PA:Z,R4-PA:ZH,R4:S,control_panel_log1,control_panel_log2,control_panel_log3,control_panel_log4,relay1_log,relay2_log,relay3_log,relay4_log,snort_log1,snort_log2,snort_log3,snort_log4,marker
0,70.399324,127673.0908,-49.572308,127648.0176,-169.578319,127723.2374,65.689611,605.91099,-57.003571,626.78553,-173.589023,602.43190,70.422243,127673.0908,0.0,0.0,0.0,0.0,65.007791,611.58740,118.567886,13.18392,-100.869220,13.91636,59.999,0.01,6.391383,0.076290,0,60.658268,124631.8125,-59.295959,124484.3594,-179.338078,124715.0703,-119.550481,612.796753,117.726752,632.532104,0.859680,...,-49.537931,127096.4056,-169.532482,127773.3839,65.643775,604.44611,-56.871791,621.84156,-173.869773,599.86836,70.462350,127522.6512,0.0,0.0,0.0,0.0,64.950496,608.47453,119.301272,12.26837,-102.060972,11.71904,59.999,0.01,6.341832,0.077897,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Natural
1,73.688102,130280.7109,-46.300719,130255.6377,-166.278082,130355.9307,71.831719,483.59351,-50.947407,500.98896,-167.487023,481.39619,73.705291,130305.7842,0.0,0.0,0.0,0.0,71.109792,488.35437,125.792884,10.62038,-95.884487,11.35282,60.005,0.00,8.185463,0.024924,0,66.069035,128277.4297,-53.885193,128126.3047,-173.932803,128354.7578,-114.925241,489.349365,122.148740,505.754456,5.468445,...,-46.249153,129704.0257,-166.232245,130381.0040,71.837448,481.76241,-50.792709,496.04499,-167.618803,477.73399,73.756857,130130.2713,0.0,0.0,0.0,0.0,71.121251,485.05839,124.274546,10.25416,-95.454769,9.70483,60.005,0.00,8.141328,0.027210,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Natural
2,73.733939,130305.7842,-46.254883,130280.7109,-166.232245,130381.0040,71.808800,483.59351,-50.913030,500.98896,-167.441186,481.02997,73.751127,130330.8575,0.0,0.0,0.0,0.0,71.132710,488.35437,125.007932,10.62038,-94.520847,11.35282,60.005,0.00,8.190006,0.027904,0,66.112980,128283.9453,-53.852234,128144.4688,-173.883366,128382.9609,-114.919736,488.885864,122.214665,505.531311,5.462952,...,-46.197587,129729.0990,-166.192138,130381.0040,71.866096,481.39619,-50.781249,496.41121,-167.590155,478.10021,73.796964,130155.3446,0.0,0.0,0.0,0.0,71.144169,485.24150,125.277222,10.25416,-95.970431,10.07105,60.005,0.00,8.162755,0.026663,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Natural
3,74.083443,130581.5902,-45.899649,130556.5169,-165.882741,130656.8100,72.152575,482.86107,-50.437475,499.15786,-167.286487,481.39619,74.106361,130581.5902,0.0,0.0,0.0,0.0,71.459296,487.62193,127.597701,9.70483,-96.657980,10.43727,60.003,0.00,8.171532,0.025617,0,66.508489,128585.4063,-53.445742,128442.6094,-173.487857,128673.4141,-114.815371,489.864349,122.648628,504.226685,5.693665,...,-45.848083,129979.8317,-165.836904,130681.8832,72.181223,481.21308,-50.311424,494.76322,-167.395349,478.28332,74.152198,130431.1505,0.0,0.0,0.0,0.0,71.482215,484.69217,126.474704,9.33861,-97.253856,9.15550,60.003,0.00,8.135292,0.026595,0.0,0,0,0,0,0,0,0,0,0,0,0,0,Natural
4,74.553268,131083.0556,-45.424094,131057.9823,-165.424375,131158.2754,72.118198,484.50906,-50.013486,497.69298,-167.464104,484.69217,74.570457,131108.1288,0.0,0.0,0.0,0.0,71.539510,488.90370,127.666456,7.50751,-99.923839,8.60617,60.001,0.00,8.079496,0.032941,0,67.035827,129106.9063,-52.912905,128974.5859,-172.949529,129196.8125,-114.919736,491.294861,123.071594,502.498627,5.413513,...,-45.372528,130506.3704,-165.372808,131183.3486,72.164034,482.86107,-49.881706,493.11523,-167.601614,481.21308,74.627753,130932.6159,0.0,0.0,0.0,0.0,71.556699

In [9]:
df['marker'].value_counts()

Attack     55663
Natural    22714
Name: marker, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78377 entries, 0 to 5235
Columns: 129 entries, R1-PA1:VH to marker
dtypes: float64(113), int64(15), object(1)
memory usage: 77.7+ MB


In [11]:
df.columns

Index(['R1-PA1:VH', 'R1-PM1:V', 'R1-PA2:VH', 'R1-PM2:V', 'R1-PA3:VH',
       'R1-PM3:V', 'R1-PA4:IH', 'R1-PM4:I', 'R1-PA5:IH', 'R1-PM5:I',
       ...
       'control_panel_log4', 'relay1_log', 'relay2_log', 'relay3_log',
       'relay4_log', 'snort_log1', 'snort_log2', 'snort_log3', 'snort_log4',
       'marker'],
      dtype='object', length=129)

In [12]:
df.describe()

,R1-PA1:VH,R1-PM1:V,R1-PA2:VH,R1-PM2:V,R1-PA3:VH,R1-PM3:V,R1-PA4:IH,R1-PM4:I,R1-PA5:IH,R1-PM5:I,R1-PA6:IH,R1-PM6:I,R1-PA7:VH,R1-PM7:V,R1-PA8:VH,R1-PM8:V,R1-PA9:VH,R1-PM9:V,R1-PA10:IH,R1-PM10:I,R1-PA11:IH,R1-PM11:I,R1-PA12:IH,R1-PM12:I,R1:F,R1:DF,R1-PA:Z,R1-PA:ZH,R1:S,R2-PA1:VH,R2-PM1:V,R2-PA2:VH,R2-PM2:V,R2-PA3:VH,R2-PM3:V,R2-PA4:IH,R2-PM4:I,R2-PA5:IH,R2-PM5:I,R2-PA6:IH,...,R4-PM1:V,R4-PA2:VH,R4-PM2:V,R4-PA3:VH,R4-PM3:V,R4-PA4:IH,R4-PM4:I,R4-PA5:IH,R4-PM5:I,R4-PA6:IH,R4-PM6:I,R4-PA7:VH,R4-PM7:V,R4-PA8:VH,R4-PM8:V,R4-PA9:VH,R4-PM9:V,R4-PA10:IH,R4-PM10:I,R4-PA11:IH,R4-PM11:I,R4-PA12:IH,R4-PM12:I,R4:F,R4:DF,R4-PA:Z,R4-PA:ZH,R4:S,control_panel_log1,control_panel_log2,control_panel_log3,control_panel_log4,relay1_log,relay2_log,relay3_log,relay4_log,snort_log1,snort_log2,snort_log3,snort_log4
count,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,7.837700e+04,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,...,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,7.837700e+04,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000,78377.000000
mean,-15.802424,130764.039577,2.175196,131035.528095,6.834315,131395.717581,-14.334996,393.949321,3.538540,387.438133,6.129781,381.912845,-15.798835,131056.980030,0.207857,297.083556,0.227606,87.397031,-14.504282,386.557188,-1.734936,9.979982,6.123374,9.494176,59.992801,0.000529,inf,0.018428,788.868750,-15.216491,127033.389923,4.751134,128015.428015,5.510410,128362.246185,15.836436,395.109497,-6.961603,392.508845,-6.437082,...,131117.453645,2.278991,131355.212680,7.065760,131745.074472,-13.931742,391.330912,3.446031,384.399819,6.096400,379.952713,-15.563852,131397.999652,0.257084,292.112647,0.207103,82.439295,-14.144585,384.036050,-1.859917,9.834635,5.989009,9.073233,59.992750,0.000602,inf,0.016616,749.014459,0.000038,0.000038,0.000026,0.000026,0.035916,0.026436,0.026500,0.035597,0.000051,0.000051,0.000089,0.000077
std,100.876750,8546.118477,111.743169,5393.135370,97.065063,5443.752388,99.601107,190.966011,109.504977,151.277183,95.294904,153.326452,100.877118,6152.379663,13.075863,2687.617199,12.488596,897.541412,99.605025,154.484403,68.383257,47.241783,73.059209,47.875569,0.610045,0.087799,NaN,0.242813,14048.448459,101.837622,16155.767175,111.043204,12106.876201,96.270117,11990.863815,99.876094,171.765698,94.996062,152.357765,108.896267,...,7984.885533,111.828597,4733.901358,97.085981,4777.648212,99.653296,187.094100,109.561785,148.882516,95.495537,150.929876,100.882320,5536.542517,13.150046,2621.155809,12.523032,850.696972,99.627784,151.746973,67.783975,47.562328,72.087423,45.998572,0.609958,0.087273,NaN,0.248023,14041.170907,0.006187,0.006187,0.005051,0.005051,0.186082,0.160430,0.160618,0.185285,0.007144,0.007144,0.009450,0.008749
min,-179.988962,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.501948,0.000000,-179.903018,0.000000,-179.994691,0.000000,-179.994691,0.000000,-179.988962,0.000000,0.000000,-4.010000,1.852102e-01,-3.140569,0.000000,-179.994691,0.000000,-179.997259,0.000000,-179.994691,0.000000,-179.997259,0.000000,-179.994691,0.000000,-179.994691,...,-174.341917,-179.997259,-169.984571,-179.994691,-178.875387,-1

In [13]:
# np.any(np.isnan(df))

In [14]:
# torch.cuda.is_available()

Data preprocessing

In [15]:
# label Encoding
df['marker'] = df['marker'].replace(['Natural'],0)
df['marker'] = df['marker'].replace(['Attack'],1)
df['marker'].value_counts()

1    55663
0    22714
Name: marker, dtype: int64

In [16]:
dataset = df.iloc[:]
missing_val_count_by_column = (dataset.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [17]:
# #normalization
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1,1))
# df_normalized=scaler.fit_transform(dataset.values.reshape(-1,1))

In [18]:
# Separate target from predictors
y = df.marker
x = df.drop(['marker'], axis=1)

In [19]:
# print(y.shape)
# print(x.shape)

In [20]:
values = x
# np.all(np.isfinite(values))
values.replace([np.inf, -np.inf], np.nan, inplace=True)


In [21]:
#normalization
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# scaler = MinMaxScaler()
scaler = StandardScaler()
X_normalized=scaler.fit_transform(values)
X_normalized

array([[ 8.54530879e-01, -3.61680979e-01, -4.63096184e-01, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03],
       [ 8.87133027e-01, -5.65557055e-02, -4.33818243e-01, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03],
       [ 8.87587412e-01, -5.36218052e-02, -4.33408045e-01, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03],
       ...,
       [ 1.42241229e-03,  3.43950999e-02, -1.23329575e+00, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03],
       [ 6.84036156e-04,  3.43950999e-02, -1.23396232e+00, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03],
       [ 5.92562950e-05,  3.43950999e-02, -1.23452634e+00, ...,
        -7.14408742e-03, -9.45092021e-03, -8.74979607e-03]])

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, y, train_size=0.9, test_size=0.1, random_state=0)

In [24]:
X_train_full.shape

(70539, 128)

In [25]:
# from keras.preprocessing.sequence import TimeseriesGenerator

In [26]:
#function to convert to time domain dataset
def create_dataset(X, time_steps):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [27]:
# reshape to [samples, time_steps, n_features]

TIME_STEPS = 1
X_train = pd.DataFrame(X_train_full)
X_train = create_dataset(X_train, TIME_STEPS)
y_train = pd.DataFrame(y_train)
y_train = create_dataset(y_train, TIME_STEPS)
X_valid = pd.DataFrame(X_valid_full)
X_valid = create_dataset(X_valid, TIME_STEPS)
y_valid = pd.DataFrame(y_valid)
y_valid = create_dataset(y_valid, TIME_STEPS)
print(X_valid.shape)
print(y_valid.shape)

(7837, 1, 128)
(7837, 1, 1)


In [28]:
# reshape input to be 3D [samples, timesteps, features]
# X_train_full = X_train_full.reshape((X_train_full.shape[0], 1, X_train_full.shape[1]))
# X_valid_full = X_valid_full.reshape((X_valid_full.shape[0], 1, X_valid_full.shape[1]))
# print(X_train_full.shape, y_train.shape, X_valid_full.shape, y_valid.shape) 

# Construct Model

In [29]:
class LSTM(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
              
        self.lstm1 = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            batch_first=False,dropout = 0)
        self.lstm2 = nn.LSTM(input_size=hidden_size, 
                            hidden_size=32,
                            num_layers=num_layers, 
                            batch_first=False,dropout = 0)
        self.dropout1 = nn.Dropout(p=0.3)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc = nn.Linear(32,output_size)
      
        self.sigmoid = nn.Sigmoid()
     
    def forward(self, x):     
        per_out=[]
        lstm_out, self.hidden_cell = self.lstm1(x)
        per_out.append(lstm_out)
        lstm_out = self.dropout1(lstm_out)
        lstm_out, self.hidden_cell = self.lstm2(lstm_out)
        per_out.append(lstm_out)
        lstm_out = self.dropout2(lstm_out)
        out = self.fc(lstm_out)
        per_out.append(out)
        score = self.sigmoid(out)
        return score, per_out

In [39]:
class LSTM(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
              
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            batch_first=False,dropout = 0)

        self.dropout = nn.Dropout(p=0.3)
        self.fc = nn.Linear(64,output_size)
      
        self.sigmoid = nn.Sigmoid()
     
    def forward(self, x):     
        per_out=[]
        lstm_out, self.hidden_cell = self.lstm(x)
        per_out.append(lstm_out)
        lstm_out = self.dropout(lstm_out)    
        out = self.fc(lstm_out)
        per_out.append(out)
        score = self.sigmoid(out)
        return score, per_out

In [30]:
def evaluate_accuracy(x,y,model):
    output,pre_out = model(x)
    output = torch.reshape(output,[-1,1])
    correct = (output.ge(0.5) == y).sum().item()
    n = y.shape[0]
    return correct/n

In [40]:
input_size = 128
output_size = 1
hidden_size = 64
num_layers = 2
device = torch.device('cuda')
model = LSTM(input_size,output_size, hidden_size, num_layers)
loss_function = nn.BCELoss()
# model = LSTM(input_size,output_size, hidden_size, num_layers)
# loss_function = nn.BCELoss()
# optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [41]:
print(model)

LSTM(
  (lstm): LSTM(128, 64, num_layers=2)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [33]:
len(X_train)

70538

In [34]:
# x = X_train_full[:2,:,:]
# x.shape

In [35]:
# loss

In [36]:
import math
math.ceil(len(X_train)/60)

1176

In [37]:
import torch.utils.data as Data
# 将输入和输出封装进Data.TensorDataset()类对象
x = torch.tensor(X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
y = torch.tensor(y)
y = y.float()
print(x.shape)
print(y.shape)

torch_dataset = Data.TensorDataset(x,y)

torch.Size([70538, 1, 128])
torch.Size([70538, 1, 1])


In [38]:
BATCH_SIZE = 60
train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Training and Evaluation

In [ ]:
epochs = 100
for epoch in range(epochs):
  for step, (batch_x, batch_y) in enumerate(train_dataloader):
    batch_x = batch_x.cuda()
   
    output,pre_out = model(batch_x)
    output = torch.reshape(output,[-1,1])

    batch_y = np.array(batch_y)
    batch_y = torch.tensor(np.reshape(batch_y,[-1,1]))
    batch_y = batch_y.float()
    batch_y = batch_y.cuda()

    loss = loss_function(output,batch_y)
    acc = evaluate_accuracy(batch_x,batch_y,model)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 5 == 0:
      print("epoch:{} Step:{} loss:{} acc:{}".format(epoch,step,loss.item(),acc))

In [50]:
epochs = 10
for epoch in range(epochs):
    model.train()
    x = torch.tensor( X_train[:,:,:]).float()
    x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
    y = y_train[:]
    y = np.array(y)
    y = torch.tensor(np.reshape(y,[-1,1]))
    y = y.float()
    
    x = x
    y = y
    
    output,pre_out = model(x)
    output = torch.reshape(output,[-1,1])
    loss = loss_function(output,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 1 == 0:
        print("epoch:{} loss:{}".format(epoch,loss.item()))

epoch:0 loss:0.700545072555542


KeyboardInterrupt: ignored

In [49]:
print(X_valid.shape)
print(y_valid.shape)
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
# x = torch.tensor(X_train)
# y = torch.tensor(y_train)

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid
y_val = y_val

train_acc = evaluate_accuracy(x_valid,y_val,model)
print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",train_acc)

(7837, 1, 128)
(7837, 1, 1)
epoch:4 loss:0.7012091279029846
train acc 0.3381723326433979
valid acc: 0.33469439836672193
